## Halloween Challenge
Find the best solution with the fewest calls to the fitness functions for:
* num_points = [100, 1_000, 5_000]
* num_sets = num_points
* density = [.3, .7]

### Local Search - Set Covering
To resolve this challenge I implemented these different **Single-State Methods**:
* Random-Mutation Hill Climbing
* Steepest Ascent Hill Climbing
* K population Random-Mutation Hill Climbing
* K population Steepest Ascent Hill Climbing
* Simulated Annealing

In [161]:
from random import random, randint, seed
from functools import reduce
from pprint import pprint
import numpy as np
from copy import copy
from itertools import product
from scipy import sparse

In [12]:
def make_set_covering_problem(num_points, num_sets, density):
    """Returns a sparse array where rows are sets and columns are the covered items"""
    seed(num_points * 2654435761 + num_sets + density)
    sets = sparse.lil_array((num_sets, num_points), dtype=bool)
    for s, p in product(range(num_sets), range(num_points)):
        if random() < density:
            sets[s, p] = True
    for p in range(num_points):
        sets[randint(0, num_sets - 1), p] = True
    return sets.toarray()

In [110]:
PROBLEM_SIZE = [100, 1_000, 5_000]
DENSITY = [0.3, 0.7]

I defined how fitness function a function that return the number of covered cells, and the negative number of sets used

In [126]:
def fitness(state, sets):
    num_true = np.sum(
        reduce(
            np.logical_or,
            [sets[i] for i, t in enumerate(state) if t],
            False,
        )
    )
    return num_true, -sum(state)

In [133]:
def generate_initial_state(prob, problem_size):
    return [True if random() < prob else False for _ in range(problem_size)]

With the following cell I checked if all the configuration of the problem are solvable and saved the sets, initial state and conf in an array problems

In [138]:
problems = []
for p in PROBLEM_SIZE:
    for d in DENSITY:
        initial_state = generate_initial_state(0.3, p)
        sets = make_set_covering_problem(p, p, d)
        assert fitness([True for _ in range(p)], sets)[0] == p, "Problem not solvable"
        problems.append((sets, initial_state, (p, d)))

I defined a tweak function that random modify one set from taken to untaken or from untaken to taken 

In [127]:
def tweak(state, problem_size):
    new_state = copy(state)
    index = randint(0, problem_size - 1)
    new_state[index] = not state[index]
    return new_state

### Random Mutation Hill Climbing

In [131]:
def RMHC(sets, problem_size, state, max_iteration):
    goodness = fitness(state, sets)
    for _ in range(max_iteration):
        new_state = tweak(state, problem_size)
        goodness_new_state = fitness(new_state, sets)
        if goodness_new_state >= goodness:
            state = new_state
            goodness = goodness_new_state
    return state, goodness, max_iteration + 1

In [140]:
for sets, initial_state, (p, d) in problems:
    solution_state, goodness, evaluations = RMHC(sets, p, initial_state, 100)
    print(
        f"Random Mutation Hill Climbing with 100 steps:\nDensity: {d}\nProblem size: {p}\nNumber of calls to fitness function: {evaluations}\nGoodness: {goodness}\n"
    )

Random Mutation Hill Climbing with 100 steps:
Density: 0.3
Problem size: 100
Number of calls to fitness function: 101
Goodness: (100, -12)

Random Mutation Hill Climbing with 100 steps:
Density: 0.7
Problem size: 100
Number of calls to fitness function: 101
Goodness: (100, -9)

Random Mutation Hill Climbing with 100 steps:
Density: 0.3
Problem size: 1000
Number of calls to fitness function: 101
Goodness: (1000, -298)

Random Mutation Hill Climbing with 100 steps:
Density: 0.7
Problem size: 1000
Number of calls to fitness function: 101
Goodness: (1000, -273)

Random Mutation Hill Climbing with 100 steps:
Density: 0.3
Problem size: 5000
Number of calls to fitness function: 101
Goodness: (5000, -1452)

Random Mutation Hill Climbing with 100 steps:
Density: 0.7
Problem size: 5000
Number of calls to fitness function: 101
Goodness: (5000, -1456)



### Steepest Ascent Hill Climbing

In [146]:
def SAHC(sets, problem_size, state, max_iteration, number_of_teawks):
    goodness = fitness(state, sets)
    for _ in range(max_iteration):
        new_state = tweak(state, problem_size)
        new_state_goodness = fitness(new_state, sets)
        for _ in range(number_of_teawks):
            tmp = tweak(state, problem_size)
            tmp_goodness = fitness(tmp, sets)
            if tmp_goodness > new_state_goodness:
                new_state = tmp
                new_state_goodness = tmp_goodness
        if new_state_goodness > goodness:
            state = new_state
            goodness = new_state_goodness
    return state, goodness, (max_iteration * number_of_teawks) + 1

In [147]:
for sets, initial_state, (p, d) in problems:
    solution_state, goodness, evaluations = SAHC(sets, p, initial_state, 100, 20)
    print(
        f"Stepest-step Hill Climbing with 100 steps and number of tweaks 20:\nDensity: {d}\nProblem size: {p}\nNumber of calls to fitness function: {evaluations}\nGoodness: {goodness}\n"
    )

Stepest-step Hill Climbing with 100 steps and number of tweaks 20:
Density: 0.3
Problem size: 100
Number of calls to fitness function: 2001
Goodness: (100, -9)

Stepest-step Hill Climbing with 100 steps and number of tweaks 20:
Density: 0.7
Problem size: 100
Number of calls to fitness function: 2001
Goodness: (100, -3)

Stepest-step Hill Climbing with 100 steps and number of tweaks 20:
Density: 0.3
Problem size: 1000
Number of calls to fitness function: 2001
Goodness: (1000, -227)

Stepest-step Hill Climbing with 100 steps and number of tweaks 20:
Density: 0.7
Problem size: 1000
Number of calls to fitness function: 2001
Goodness: (1000, -201)

Stepest-step Hill Climbing with 100 steps and number of tweaks 20:
Density: 0.3
Problem size: 5000
Number of calls to fitness function: 2001
Goodness: (5000, -1380)

Stepest-step Hill Climbing with 100 steps and number of tweaks 20:
Density: 0.7
Problem size: 5000
Number of calls to fitness function: 2001
Goodness: (5000, -1388)



### K Population Random Mutation Hill Climbing

In [155]:
def k_population_RMHC(sets, p, max_iteration, k):
    number_of_evaluations = 0
    solution_states = []
    goodness_states = []
    for _ in range(k):
        state = generate_initial_state(0.3, p)
        solution_state, goodness, evaluations = RMHC(sets, p, state, max_iteration)
        solution_states.append(solution_state)
        goodness_states.append(goodness)
        number_of_evaluations += evaluations

    solution_goodness = max(goodness_states)
    return solution_states[goodness_states.index(solution_goodness)], solution_goodness, number_of_evaluations

In [156]:
for sets, _, (p, d) in problems:
    solution_state, goodness, evaluations = k_population_RMHC(sets, p, 100, 5)
    print(
        f"Random Mutation Hill Climbing with 100 steps and 5 random population:\nDensity: {d}\nProblem size: {p}\nNumber of calls to fitness function: {evaluations}\nGoodness: {goodness}\n"
    )

Random Mutation Hill Climbing with 100 steps and 5 random population:
Density: 0.3
Problem size: 100
Number of calls to fitness function: 505
Goodness: (100, -9)

Random Mutation Hill Climbing with 100 steps and 5 random population:
Density: 0.7
Problem size: 100
Number of calls to fitness function: 505
Goodness: (100, -7)

Random Mutation Hill Climbing with 100 steps and 5 random population:
Density: 0.3
Problem size: 1000
Number of calls to fitness function: 505
Goodness: (1000, -262)

Random Mutation Hill Climbing with 100 steps and 5 random population:
Density: 0.7
Problem size: 1000
Number of calls to fitness function: 505
Goodness: (1000, -257)

Random Mutation Hill Climbing with 100 steps and 5 random population:
Density: 0.3
Problem size: 5000
Number of calls to fitness function: 505
Goodness: (5000, -1433)

Random Mutation Hill Climbing with 100 steps and 5 random population:
Density: 0.7
Problem size: 5000
Number of calls to fitness function: 505
Goodness: (5000, -1423)



### K Population Steepest Ascent Hill Climbing

In [157]:
def k_population_SAHC(sets, p, max_iteration, k, n):
    number_of_evaluations = 0
    solution_states = []
    goodness_states = []
    for _ in range(k):
        state = generate_initial_state(0.3, p)
        solution_state, goodness, evaluations = SAHC(sets, p, state, max_iteration, n)
        solution_states.append(solution_state)
        goodness_states.append(goodness)
        number_of_evaluations += evaluations

    solution_goodness = max(goodness_states)
    return solution_states[goodness_states.index(solution_goodness)], solution_goodness, number_of_evaluations

In [160]:
for sets, _, (p, d) in problems:
    solution_state, goodness, evaluations = k_population_SAHC(sets, p, 100, 20, 5)
    print(
        f"Stepest-step Hill Climbing with 100 steps, number of tweaks 20 and 5 random population:\nDensity: {d}\nProblem size: {p}\nNumber of calls to fitness function: {evaluations}\nGoodness: {goodness}\n"
    )

Stepest-step Hill Climbing with 100 steps, number of tweaks 20 and 5 random population:
Density: 0.3
Problem size: 100
Number of calls to fitness function: 10020
Goodness: (100, -7)

Stepest-step Hill Climbing with 100 steps, number of tweaks 20 and 5 random population:
Density: 0.7
Problem size: 100
Number of calls to fitness function: 10020
Goodness: (100, -3)

Stepest-step Hill Climbing with 100 steps, number of tweaks 20 and 5 random population:
Density: 0.3
Problem size: 1000
Number of calls to fitness function: 10020
Goodness: (1000, -195)

Stepest-step Hill Climbing with 100 steps, number of tweaks 20 and 5 random population:
Density: 0.7
Problem size: 1000
Number of calls to fitness function: 10020
Goodness: (1000, -197)

Stepest-step Hill Climbing with 100 steps, number of tweaks 20 and 5 random population:
Density: 0.3
Problem size: 5000
Number of calls to fitness function: 10020
Goodness: (5000, -1370)

Stepest-step Hill Climbing with 100 steps, number of tweaks 20 and 5 ran

### Simulated Annealing

In [178]:
def simulated_annealing(sets, problem_size, state, max_iteration, t):
    goodness = fitness(state, sets)
    for _ in range(max_iteration):
        new_state = tweak(state, problem_size)
        goodness_new_state = fitness(new_state, sets)
        if goodness_new_state >= goodness or random() > np.exp((goodness[1] - goodness_new_state[1]) / t):
            state = new_state
            goodness = goodness_new_state
        t = t / 10
    return state, goodness, max_iteration + 1

In [179]:
for sets, initial_state, (p, d) in problems:
    solution_state, goodness, evaluations = simulated_annealing(sets, p, initial_state, 100, 25)
    print(
        f"Random Mutation Hill Climbing with 100 steps:\nDensity: {d}\nProblem size: {p}\nNumber of calls to fitness function: {evaluations}\nGoodness: {goodness}\n"
    )

/tmp/ipykernel_3601/2618015769.py:6: RuntimeWarning: overflow encountered in exp
  if goodness_new_state >= goodness or random() > np.exp((goodness[1] - goodness_new_state[1]) / t):


Random Mutation Hill Climbing with 100 steps:
Density: 0.3
Problem size: 100
Number of calls to fitness function: 101
Goodness: (100, -13)

Random Mutation Hill Climbing with 100 steps:
Density: 0.7
Problem size: 100
Number of calls to fitness function: 101
Goodness: (100, -8)

Random Mutation Hill Climbing with 100 steps:
Density: 0.3
Problem size: 1000
Number of calls to fitness function: 101
Goodness: (1000, -303)

Random Mutation Hill Climbing with 100 steps:
Density: 0.7
Problem size: 1000
Number of calls to fitness function: 101
Goodness: (1000, -269)

Random Mutation Hill Climbing with 100 steps:
Density: 0.3
Problem size: 5000
Number of calls to fitness function: 101
Goodness: (5000, -1455)

Random Mutation Hill Climbing with 100 steps:
Density: 0.7
Problem size: 5000
Number of calls to fitness function: 101
Goodness: (5000, -1457)

